# Pandas Express
# TTC-Delays project
Team Members: 

Gaspare Castagna

Amirhossein Heidari

Avery Hoffer

Michael Metallo

In [1]:
import numpy as numpy
import pandas as pd
import os
import numpy as np
!pip install python-Levenshtein
import Levenshtein

Import, read, concat bus delay data

In [2]:
def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    data_frames = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [3]:
df_2014 = read_excel_sheets('bus_2014.xlsx')
df_2015 = read_excel_sheets('bus_2015.xlsx') 
df_2016 = read_excel_sheets('bus_2016.xlsx') 
df_2017 = read_excel_sheets('bus_2017.xlsx') 
df_2018 = read_excel_sheets('bus_2018.xlsx')
df_2018 = df_2018.iloc[:, :-1]
df_2019 = read_excel_sheets('bus_2019.xlsx')
df_2019 = df_2019.iloc[:, :-3]
df_2020 = read_excel_sheets('bus_2020.xlsx')
df_2020 = df_2020.iloc[:, :-3]
df_2020.rename(columns={'Gap': 'Min Gap', 'Delay': 'Min Delay'}, inplace=True)
df_2021 = read_excel_sheets('bus_2021.xlsx')
df_2021.rename(columns={'Date': 'Report Date'}, inplace=True)
df_2021 = df_2021.iloc[:, :-3]
df_2022 = read_excel_sheets('bus_2022.xlsx')
df_2022.rename(columns={'Date': 'Report Date'}, inplace=True)

bus_delay_list = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022]
df_bus_delay = pd.concat(bus_delay_list)

df_bus_delay['YTime'] = pd.to_datetime(df_bus_delay['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M')
df_bus_delay['DateTime'] = df_bus_delay['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ df_bus_delay['YTime']
df_bus_delay['DateTime'] = pd.to_datetime(df_bus_delay['DateTime'])

drop_columns = ['YTime','Time','Report Date','Direction']
df_bus_delay = df_bus_delay.drop(drop_columns, axis = 1)
df_bus_delay = df_bus_delay.dropna(subset=['Route'])

df_bus_delay['copy'] = df_bus_delay['Location'].str.lower()
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('vp','victoria park')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace(' to ','$').str.replace('b/w','').str.replace('w/b','').str.replace('e/b','').str.replace('s/b','').str.replace('n/b','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('stn','station').str.replace(' and ','&').str.replace('/','&').str.replace(' ','')
df_bus_delay['copy'] = df_bus_delay['copy'].astype(str)
df_bus_delay = df_bus_delay[~df_bus_delay['copy'].str.isdigit()]
df_bus_delay['copy'] = df_bus_delay['copy'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

In [4]:
df_bus_delay.nunique()

Route           491
Day               7
Location     117707
Incident         29
Min Delay       970
Min Gap        1003
Vehicle        4300
DateTime     368729
copy          75884
dtype: int64

In [176]:
print("2014: "+str(df_2014['Time'][1]))
print("2015: "+str(df_2015['Time'][1]))
print("2016: "+str(df_2016['Time'][1]))
print("2017: "+str(df_2017['Time'][1]))
print("2018: "+str(df_2018['Time'][1]))
print("2019: "+str(df_2019['Time'][1]))
print("2020: "+str(df_2020['Time'][1]))
print("2021: "+str(df_2021['Time'][1]))
print("2022: "+str(df_2022['Time'][1]))

2014: 00:55:00
2015: 00:28:00
2016: 00:25:00
2017: 01:02:00
2018: 00:20:00
2019: 00:15:00
2020: 01:31:00
2021: 05:04
2022: 02:00


In [6]:

df_bus_delay.head()

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark


In the section below I am testing the fuzz score function we will be using:

In [4]:
stops = pd.read_csv('stops.txt')
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,262,662,Danforth Rd at Kennedy Rd,NaN,43.714379,-79.260939,NaN,NaN,NaN,NaN,NaN,1
1,263,929,Davenport Rd at Bedford Rd,NaN,43.674448,-79.399659,NaN,NaN,NaN,NaN,NaN,1
2,264,940,Davenport Rd at Dupont St,NaN,43.675511,-79.401938,NaN,NaN,NaN,NaN,NaN,2
3,265,1871,Davisville Ave at Cleveland St,NaN,43.702088,-79.378112,NaN,NaN,NaN,NaN,NaN,1
4,266,11700,Disco Rd at Attwell Dr,NaN,43.701362,-79.594843,NaN,NaN,NaN,NaN,NaN,1


In [218]:
stops['stop_name'].nunique()

7594

In [5]:
stops['cleaned_stops'] = stops['stop_name'].str.replace('at', '&').str.replace('Rd', '').str.replace('Ave', '').str.replace('Dr', '').str.replace('St', '')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace(' ', '').str.lower()

In [74]:
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,cleaned_stops
0,262,662,Danforth Rd at Kennedy Rd,NaN,43.714379,-79.260939,NaN,NaN,NaN,NaN,NaN,1,danforth&kennedy
1,263,929,Davenport Rd at Bedford Rd,NaN,43.674448,-79.399659,NaN,NaN,NaN,NaN,NaN,1,davenport&bedford
2,264,940,Davenport Rd at Dupont St,NaN,43.675511,-79.401938,NaN,NaN,NaN,NaN,NaN,2,davenport&dupont
3,265,1871,Davisville Ave at Cleveland St,NaN,43.702088,-79.378112,NaN,NaN,NaN,NaN,NaN,1,davisville&cleveland
4,266,11700,Disco Rd at Attwell Dr,NaN,43.701362,-79.594843,NaN,NaN,NaN,NaN,NaN,1,disco&attwell


This function takes in a string "location" and compares it to each entry in a list of strings "entry". 
It returns a string corresponding to the "best" match for the input string from the list of strings.
We can adjust the threshold to avoid false positives. Strings with no matches above the threshold will not be modified.

In [220]:
def compute_similarity(string1, string2):
    distance = Levenshtein.distance(string1, string2)
    max_length = max(len(string1), len(string2))
    if max_length == 0:
        return 100
    else:
        return 100 - (distance / max_length) * 100
    
def synchronize_locations_ls(location, reference_list):
    # 1) create an array with the similarity scores between the location and each entry in reference list
    similarity_scores = []
    for item in reference_list:
        string=str(item)
        print(location)
        print(string)
        print(compute_similarity(location[0], string))
        similarity_scores.append(compute_similarity(location, string))
    # 2) find the highest score in this list
    max_score = max(similarity_scores)
    # 3) if the score is above the threshold, return the string from the list that corresponds to the highest score
    interpolated = reference_list[similarity_scores.index(max_score)] if max_score <= 70 else "user check"
    return interpolated

In [6]:
from fuzzywuzzy import process

def find_closest_match(row, reference_list):
    match = process.extractOne(row, reference_list)
    print('completed one row')
    return match[0]

In [11]:
location = 'yorkmills'
reference_list = ['yonge&steeles', 'yorkmillsstation','victoriapark&danforth']

In [12]:
print(find_closest_match(location, reference_list))

completed one row
yorkmillsstation


In [11]:
df_bus_delay['synchronized'] = df_bus_delay['copy'].apply(find_closest_match, args=(stops['cleaned_stops'],))

completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed one row
completed 

KeyboardInterrupt: 

In [194]:
print(compute_similarity('youngandsteales','yongeandsteeles'))

80.0


from fuzzywuzzy import fuzz

def synchronize_locations_fuzz(location, reference_list):
    # 1) Create an array with the similarity scores between the location and each entry in the reference list
    similarity_scores = [fuzz.ratio(location, string) for string in reference_list]
    # 2) Find the highest score in this list
    max_score = max(similarity_scores)
    # 3) If the score is above the threshold, return the string from the list that corresponds to the highest score
    interpolated = reference_list[similarity_scores.index(max_score)] if max_score >= 40 else "user check"
    return interpolated

In [210]:
locations = ['yorkmillsstation']
reference = ['yonge&steeles', 'yorkmillsstation', 'davenport&dufferin']

In [211]:
synchronized_values = synchronize_locations_ls(locations, reference)

['yorkmillsstation']
yonge&steeles
25.0
['yorkmillsstation']
yorkmillsstation
100.0
['yorkmillsstation']
davenport&dufferin
5.555555555555557


In [208]:
synchronized_values

'yonge&steeles'

In [221]:
np_copy = list(df_bus_delay['copy'])
np_stops = list(stops['cleaned_stops'])

In [222]:
synchronized_values = synchronize_locations_ls(np_copy, np_stops)
df_bus_delay['synchronized'] = synchronized_values

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [95]:
df_bus_delay.head(50)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,synchronized
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,user check
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,user check
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,user check
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,user check
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,user check
5,129,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown,user check
6,36,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot,user check
7,53,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,markham&passmore,user check
8,36,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation,user check
9,320,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge,user check


In [82]:
def synchronize_locations_np(location, reference_list):
    synchronized_list = []
    for loc in location:
        fuzz_list = np.array([fuzz.ratio(loc, string) for string in reference_list])
        max_index = np.argmax(fuzz_list)
        if fuzz_list[max_index] >= 65:
            interpolated = reference_list[max_index]
        else:
            interpolated = "user check"
        synchronized_list.append(interpolated)
    return synchronized_list

In [83]:
synchronized_values = synchronize_locations_np(df_bus_delay['copy'].to_numpy(), stops['cleaned_stops'].to_numpy())
df_bus_delay['synchronized'] = synchronized_values

KeyboardInterrupt: 

In [75]:
def synchronize_locations(location, reference_list):
    fuzz_list = [fuzz.ratio(location, string) for string in reference_list]
    max_index = max(range(len(fuzz_list)), key=fuzz_list.__getitem__)
    interpolated = reference_list[max_index] if fuzz_list[max_index] >= 40 else "user check"
    return interpolated

In [56]:
location = "Vp&danforth"
reference_list = ['yonge&steeles','yorkmillsstation','kiplingstation','victoriapark&danforth']
print(synchronize_locations(location, reference_list))

victoriapark&danforth


In [76]:
df_bus_delay['synchronized'] = df_bus_delay['copy'].apply(synchronize_locations, reference_list=stops['cleaned_stops'])

KeyboardInterrupt: 

In [59]:
df_bus_delay.head(20)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,synchronized
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,yorkmillsstation
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,user check
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,user check
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,kiplingstation
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,victoriapark&danforth
5,129,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown,user check
6,36,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot,user check
7,53,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,markham&passmore,user check
8,36,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation,kiplingstation
9,320,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge,user check
